In [8]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [10]:
# read in with pandas
winter1 = pd.read_csv ('AFN20-21data.txt')
winter2 = pd.read_csv ('AFN21-22data.txt')
winter3 = pd.read_csv ('AFN22-23data.txt')

In [12]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [14]:
# mask out days without precipitation
codes = ['GR','GS','IC','PL','PY','RA','SG','SH','SN','SQ','TS','UP']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [16]:
precip_days2

,station,datetime,precip(mm),wxcodes
33,AFN,2021-12-02 07:52:00,T,-SN
34,AFN,2021-12-02 08:32:00,T,-SN
35,AFN,2021-12-02 08:47:00,T,-SN
36,AFN,2021-12-02 08:52:00,T,-SN
37,AFN,2021-12-02 09:04:00,T,-SN
...,...,...,...,...
3183,AFN,2022-02-25 23:52:00,T,-SN
3184,AFN,2022-02-26 00:08:00,T,-SN
3185,AFN,2022-02-26 00:23:00,T,-SN
3193,AFN,2022-02-26 05:10:00,T,-SN


In [18]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_18076\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_18076\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_18076\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [20]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_18076\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_18076\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_18076\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [22]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 52]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 52]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 52]

In [24]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [26]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [28]:
afn = LocationInfo(
    'Jaffrey', 'New Hampshire', 
    latitude=42.8142,  
    longitude=-72.0223, 
    timezone='America/New_Hampshire'
)

afn

LocationInfo(name='Jaffrey', region='New Hampshire', timezone='America/New_Hampshire', latitude=42.8142, longitude=-72.0223)

In [30]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(afn.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [32]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(afn.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [34]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(afn.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [36]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 06:59:50  2020-12-01 16:14:38  2020-12-01
1   2020-12-02 07:00:53  2020-12-02 16:14:22  2020-12-02
2   2020-12-03 07:01:56  2020-12-03 16:14:08  2020-12-03
3   2020-12-04 07:02:57  2020-12-04 16:13:57  2020-12-04
4   2020-12-05 07:03:56  2020-12-05 16:13:48  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:31:41  2021-02-24 17:31:23  2021-02-24
86  2021-02-25 06:30:06  2021-02-25 17:32:39  2021-02-25
87  2021-02-26 06:28:31  2021-02-26 17:33:54  2021-02-26
88  2021-02-27 06:26:54  2021-02-27 17:35:10  2021-02-27
89  2021-02-28 06:25:17  2021-02-28 17:36:24  2021-02-28

[90 rows x 3 columns]


In [38]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 06:59:34  2021-12-01 16:14:42  2021-12-01
1   2021-12-02 07:00:38  2021-12-02 16:14:25  2021-12-02
2   2021-12-03 07:01:41  2021-12-03 16:14:11  2021-12-03
3   2021-12-04 07:02:42  2021-12-04 16:13:59  2021-12-04
4   2021-12-05 07:03:42  2021-12-05 16:13:50  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:32:03  2022-02-24 17:31:05  2022-02-24
86  2022-02-25 06:30:29  2022-02-25 17:32:21  2022-02-25
87  2022-02-26 06:28:54  2022-02-26 17:33:36  2022-02-26
88  2022-02-27 06:27:18  2022-02-27 17:34:51  2022-02-27
89  2022-02-28 06:25:41  2022-02-28 17:36:06  2022-02-28

[90 rows x 3 columns]


In [40]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:59:19  2022-12-01 16:14:46  2022-12-01
1   2022-12-02 07:00:23  2022-12-02 16:14:29  2022-12-02
2   2022-12-03 07:01:26  2022-12-03 16:14:14  2022-12-03
3   2022-12-04 07:02:27  2022-12-04 16:14:02  2022-12-04
4   2022-12-05 07:03:28  2022-12-05 16:13:52  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:32:26  2023-02-24 17:30:47  2023-02-24
86  2023-02-25 06:30:52  2023-02-25 17:32:02  2023-02-25
87  2023-02-26 06:29:17  2023-02-26 17:33:18  2023-02-26
88  2023-02-27 06:27:41  2023-02-27 17:34:33  2023-02-27
89  2023-02-28 06:26:04  2023-02-28 17:35:48  2023-02-28

[90 rows x 3 columns]


In [42]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

C:\Users\alyha\AppData\Local\Temp\ipykernel_18076\762931898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['date'] = precip_days2['datetime'].dt.date


In [44]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,AFN,2020-12-05 11:52:00,1.5200,-RA BR,2020-12-05,2020-12-05 07:03:56,2020-12-05 16:13:48
1,AFN,2020-12-05 12:52:00,1.7800,-RA BR,2020-12-05,2020-12-05 07:03:56,2020-12-05 16:13:48
2,AFN,2020-12-05 13:52:00,2.5400,RA BR,2020-12-05,2020-12-05 07:03:56,2020-12-05 16:13:48
3,AFN,2020-12-05 14:52:00,2.0300,-SN BR,2020-12-05,2020-12-05 07:03:56,2020-12-05 16:13:48
4,AFN,2020-12-05 15:52:00,3.8100,+SN FG,2020-12-05,2020-12-05 07:03:56,2020-12-05 16:13:48
...,...,...,...,...,...,...,...
236,AFN,2021-02-27 13:52:00,0.2500,-SN BR,2021-02-27,2021-02-27 06:26:54,2021-02-27 17:35:10
237,AFN,2021-02-27 14:52:00,0.2500,-SN BR,2021-02-27,2021-02-27 06:26:54,2021-02-27 17:35:10
238,AFN,2021-02-27 16:52:00,0.0001,UP BR,2021-02-27,2021-02-27 06:26:54,2021-02-27 17:35:10
239,AFN,2021-02-27 17:52:00,1.0200,-RA BR,2021-02-27,2021-02-27 06:26:54,2021-02-27 17:35:10


In [46]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,AFN,2021-12-02 07:52:00,0.0001,-SN,2021-12-02,2021-12-02 07:00:38,2021-12-02 16:14:25
1,AFN,2021-12-02 08:52:00,0.0001,-SN,2021-12-02,2021-12-02 07:00:38,2021-12-02 16:14:25
2,AFN,2021-12-02 09:52:00,0.0001,-FZRA,2021-12-02,2021-12-02 07:00:38,2021-12-02 16:14:25
3,AFN,2021-12-02 10:52:00,1.5200,-RA BR,2021-12-02,2021-12-02 07:00:38,2021-12-02 16:14:25
4,AFN,2021-12-02 12:52:00,1.0200,-RA BR,2021-12-02,2021-12-02 07:00:38,2021-12-02 16:14:25
...,...,...,...,...,...,...,...
249,AFN,2022-02-25 19:52:00,0.7600,-SN BR,2022-02-25,2022-02-25 06:30:29,2022-02-25 17:32:21
250,AFN,2022-02-25 20:52:00,0.5100,-SN,2022-02-25,2022-02-25 06:30:29,2022-02-25 17:32:21
251,AFN,2022-02-25 21:52:00,0.0001,-SN BR,2022-02-25,2022-02-25 06:30:29,2022-02-25 17:32:21
252,AFN,2022-02-25 22:52:00,0.0001,-SN,2022-02-25,2022-02-25 06:30:29,2022-02-25 17:32:21


In [48]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,AFN,2022-12-03 14:52:00,1.0200,-RA,2022-12-03,2022-12-03 07:01:26,2022-12-03 16:14:14
1,AFN,2022-12-03 15:52:00,1.7800,RA BR,2022-12-03,2022-12-03 07:01:26,2022-12-03 16:14:14
2,AFN,2022-12-03 16:52:00,2.2900,RA BR,2022-12-03,2022-12-03 07:01:26,2022-12-03 16:14:14
3,AFN,2022-12-03 17:52:00,2.0300,-RA BR,2022-12-03,2022-12-03 07:01:26,2022-12-03 16:14:14
4,AFN,2022-12-03 18:52:00,2.5400,RA BR,2022-12-03,2022-12-03 07:01:26,2022-12-03 16:14:14
...,...,...,...,...,...,...,...
287,AFN,2023-02-26 16:52:00,0.0001,-SN,2023-02-26,2023-02-26 06:29:17,2023-02-26 17:33:18
288,AFN,2023-02-26 19:52:00,0.0001,-SN,2023-02-26,2023-02-26 06:29:17,2023-02-26 17:33:18
289,AFN,2023-02-26 20:52:00,0.0001,-SN,2023-02-26,2023-02-26 06:29:17,2023-02-26 17:33:18
290,AFN,2023-02-26 22:52:00,0.0001,-SN,2023-02-26,2023-02-26 06:29:17,2023-02-26 17:33:18


In [50]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,AFN,2020-12-05 11:52:00,1.5200,-RA BR,2020-12-05,2020-12-05 07:03:56,2020-12-05 16:13:48
1,AFN,2020-12-05 12:52:00,1.7800,-RA BR,2020-12-05,2020-12-05 07:03:56,2020-12-05 16:13:48
2,AFN,2020-12-05 13:52:00,2.5400,RA BR,2020-12-05,2020-12-05 07:03:56,2020-12-05 16:13:48
3,AFN,2020-12-05 14:52:00,2.0300,-SN BR,2020-12-05,2020-12-05 07:03:56,2020-12-05 16:13:48
4,AFN,2020-12-05 15:52:00,3.8100,+SN FG,2020-12-05,2020-12-05 07:03:56,2020-12-05 16:13:48
...,...,...,...,...,...,...,...
234,AFN,2021-02-27 11:52:00,0.2500,-SN BR,2021-02-27,2021-02-27 06:26:54,2021-02-27 17:35:10
235,AFN,2021-02-27 12:52:00,0.5100,-SN BR,2021-02-27,2021-02-27 06:26:54,2021-02-27 17:35:10
236,AFN,2021-02-27 13:52:00,0.2500,-SN BR,2021-02-27,2021-02-27 06:26:54,2021-02-27 17:35:10
237,AFN,2021-02-27 14:52:00,0.2500,-SN BR,2021-02-27,2021-02-27 06:26:54,2021-02-27 17:35:10


In [52]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_18076\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_18076\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_18076\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [54]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
0,AFN,2020-12-05 11:52:00,1.5200,-RA BR,2020-12-05,2020-12-05 07:03:56,2020-12-05 16:13:48,2020-12-05 08:03:56,2020-12-05 15:13:48
1,AFN,2020-12-05 12:52:00,1.7800,-RA BR,2020-12-05,2020-12-05 07:03:56,2020-12-05 16:13:48,2020-12-05 08:03:56,2020-12-05 15:13:48
2,AFN,2020-12-05 13:52:00,2.5400,RA BR,2020-12-05,2020-12-05 07:03:56,2020-12-05 16:13:48,2020-12-05 08:03:56,2020-12-05 15:13:48
3,AFN,2020-12-05 14:52:00,2.0300,-SN BR,2020-12-05,2020-12-05 07:03:56,2020-12-05 16:13:48,2020-12-05 08:03:56,2020-12-05 15:13:48
18,AFN,2020-12-09 14:52:00,0.0001,-SN,2020-12-09,2020-12-09 07:07:40,2020-12-09 16:13:37,2020-12-09 08:07:40,2020-12-09 15:13:37
...,...,...,...,...,...,...,...,...,...
233,AFN,2021-02-27 10:52:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:26:54,2021-02-27 17:35:10,2021-02-27 07:26:54,2021-02-27 16:35:10
234,AFN,2021-02-27 11:52:00,0.2500,-SN BR,2021-02-27,2021-02-27 06:26:54,2021-02-27 17:35:10,2021-02-27 07:26:54,2021-02-27 16:35:10
235,AFN,2021-02-27 12:52:00,0.5100,-SN BR,2021-02-27,2021-02-27 06:26:54,2021-02-27 17:35:10,2021-02-27 07:26:54,2021-02-27 16:35:10
236,AFN,2021-02-27 13:52:00,0.2500,-SN BR,2021-02-27,2021-02-27 06:26:54,2021-02-27 17:35:10,2021-02-27 07:26:54,2021-02-27 16:35:10


In [56]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [58]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [60]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 1.6614514285714288
2021-2022: 1.2473633333333334
2022-2023: 0.9374893333333334


In [62]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
5,AFN,2020-12-05 16:52:00,2.7900,+SN FG,2020-12-05,2020-12-05 07:03:56,2020-12-05 16:13:48
6,AFN,2020-12-05 17:52:00,4.8300,+SN FG,2020-12-05,2020-12-05 07:03:56,2020-12-05 16:13:48
7,AFN,2020-12-05 18:52:00,3.5600,+SN FG,2020-12-05,2020-12-05 07:03:56,2020-12-05 16:13:48
8,AFN,2020-12-05 19:52:00,2.7900,-SN BR,2020-12-05,2020-12-05 07:03:56,2020-12-05 16:13:48
9,AFN,2020-12-05 20:52:00,2.2900,-SN BR,2020-12-05,2020-12-05 07:03:56,2020-12-05 16:13:48
...,...,...,...,...,...,...,...
230,AFN,2021-02-22 23:52:00,2.2900,-SN BR,2021-02-22,2021-02-22 06:34:47,2021-02-22 17:28:51
231,AFN,2021-02-23 22:52:00,0.0001,-SN,2021-02-23,2021-02-23 06:33:14,2021-02-23 17:30:07
232,AFN,2021-02-23 23:52:00,0.0001,-SN,2021-02-23,2021-02-23 06:33:14,2021-02-23 17:30:07
239,AFN,2021-02-27 17:52:00,1.0200,-RA BR,2021-02-27,2021-02-27 06:26:54,2021-02-27 17:35:10


In [64]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_18076\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_18076\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_18076\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [66]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [68]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [70]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.7002689655172414
2021-2022: 0.6080714285714286
2022-2023: 0.5650329545454545
